<a href="https://colab.research.google.com/drive/1vUSgzd9YINzdIAdJVQjZSiKeOUAFCbp1?usp=sharing" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

In [1]:
#1: Install libraries needed for finetuning
!pip install -q unsloth "torch>=2.1" transformers accelerate peft datasets bitsandbytes
!pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#2: Load Phi-3 (medium) model with Unsloth + 4bit quantization
import torch
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="microsoft/phi-2",    # Using Phi-2 because Phi-3 is restricted, Phi-2 is public and similar size (~2.7B)
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
    use_rslora=False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch SmolVLMForConditionalGeneration forward function.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.1: Fast Phi patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

microsoft/phi-2 does not have a padding token! Will use pad_token = <|endoftext|>.
Unsloth: Making `model.base_model.model.model` require gradients


In [3]:
#3: Load CodeParrot-clean dataset (1% subset for faster training)
from datasets import load_dataset

dataset = load_dataset("lvwerra/codeparrot-clean", split="train[:1%]")
print(f"Loaded {len(dataset)} examples")

README.md:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/54 [00:00<?, ?it/s]

file-000000000001.json.gz:   0%|          | 0.00/246M [00:00<?, ?B/s]

file-000000000002.json.gz:   0%|          | 0.00/248M [00:00<?, ?B/s]

file-000000000003.json.gz:   0%|          | 0.00/247M [00:00<?, ?B/s]

...

file-000000000052.json.gz:   0%|          | 0.00/234M [00:00<?, ?B/s]

file-000000000053.json.gz:   0%|          | 0.00/230M [00:00<?, ?B/s]

file-000000000054.json.gz:   0%|          | 0.00/142M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loaded 53614 examples


In [4]:
#4: Format CodeParrot examples into ChatML instruction style
def format_chatml(example):
    user_prompt = "Write Python code for the following task:"
    return {
        "prompt": f"<|im_start|>user\n{user_prompt}\n<|im_end|>\n<|im_start|>assistant\n",
        "output": f"{example['content']}<|im_end|>"
    }

dataset = dataset.map(format_chatml)


Map:   0%|          | 0/53614 [00:00<?, ? examples/s]

In [5]:
#5: Create SupervisedDataset for finetuning
from torch.utils.data import Dataset

class SupervisedDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        prompt = self.dataset[idx]["prompt"]
        output = self.dataset[idx]["output"]
        full_prompt = prompt + output

        tokenized = self.tokenizer(full_prompt, truncation=True, padding="max_length", max_length=2048, return_tensors="pt")
        input_ids = tokenized.input_ids.squeeze(0)
        labels = input_ids.clone()
        return {"input_ids": input_ids, "labels": labels}

train_dataset = SupervisedDataset(dataset, tokenizer)


In [6]:
#6: Setup Huggingface TrainingArguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./phi3-coding-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    warmup_steps=10,
    max_steps=100,
    learning_rate=2e-4,
    fp16=True,
    bf16=False,
    logging_steps=10,
    save_strategy="no",
    report_to="none",
)


In [7]:
#7: Fine-tune with SFTTrainer
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    args=training_args,
    tokenizer=tokenizer,
    packing=False,
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 53,614 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 7,864,320/1,529,256,960 (0.51% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,1.061800
20,0.975700
30,1.176100
40,0.958100
50,0.916100
60,0.864100
70,0.914700
80,1.033400
90,0.901900
100,0.962100


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=100, training_loss=0.9764025688171387, metrics={'train_runtime': 342.9679, 'train_samples_per_second': 0.292, 'train_steps_per_second': 0.292, 'total_flos': 3264277905408000.0, 'train_loss': 0.9764025688171387})

In [8]:
#8: Save fine-tuned model and tokenizer
model.save_pretrained("./phi-coding-finetuned-model")
tokenizer.save_pretrained("./phi-coding-finetuned-model")

('./phi-coding-finetuned-model/tokenizer_config.json',
 './phi-coding-finetuned-model/special_tokens_map.json',
 './phi-coding-finetuned-model/vocab.json',
 './phi-coding-finetuned-model/merges.txt',
 './phi-coding-finetuned-model/added_tokens.json',
 './phi-coding-finetuned-model/tokenizer.json')

In [13]:
#9: Define inference function to generate responses
def chat_response(prompt, max_new_tokens=256):
    chat_prompt = f"<|im_start|>user\n{prompt}\n<|im_end|>\n<|im_start|>assistant\n"
    inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    start = decoded.find("<|im_start|>assistant\n")
    if start != -1:
        response = decoded[start + len("<|im_start|>assistant\n"):].strip()
    else:
        response = decoded.strip()
    response = response.split("<|im_end|>")[0].strip()
    return response

# Test with coding prompts
test_prompts = [

    "Explain the difference between a set and a dictionary in Python.",


]

for i, prompt in enumerate(test_prompts, 1):
    print(f"Prompt {i}: {prompt}")
    print(f"Response:\n{chat_response(prompt)}")

Prompt 1: Explain the difference between a set and a dictionary in Python.
Response:
A set is an unordered collection of unique elements, while a dictionary is a collection of key-value pairs.
